In [1]:
import os
#import jovian
import matplotlib
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [2]:
merged_df = pd.read_csv('merged_data.csv')

In [3]:
submission_df = pd.read_csv('sample submission 3.csv')

In [4]:
submission_df.nunique()

MAGIC_KEY    5379
MEAT            1
dtype: int64

In [5]:
merged_df['PURCHASE_DATE'] = pd.to_datetime(merged_df['PURCHASE_DATE'])
merged_df['LAST_PURCHASE_DATE'] = pd.to_datetime(merged_df['LAST_PURCHASE_DATE'])

### Encoding Categorical Data

In [6]:
from sklearn.preprocessing import LabelEncoder
import pickle

In [7]:
le = LabelEncoder() #creating an instance of LabelEncoder
merged_df['MAGIC_KEY_ENCODED'] = le.fit_transform(merged_df['MAGIC_KEY'])
merged_df['DELIVERY_ENCODED'] = le.fit_transform(merged_df['DELIVERY_OPTION'])
merged_df['QUALITY_ENCODED'] = le.fit_transform(merged_df['QUALITY'])

In [8]:
prb_df_without_encode = pd.read_csv('problem 3.csv')
prb_df_encode = prb_df_without_encode.copy()

In [9]:
prb_df_encode = prb_df_encode.merge(merged_df, on='MAGIC_KEY', how='left').drop_duplicates(subset=['MAGIC_KEY'])


In [10]:
#prb_df_encode.MAGIC_KEY.value_counts()

In [11]:
#prb_df_encode

## Added magic key encode in the submission df

In [12]:
prb_df_encode['MAGIC_KEY_ENCODED'] = le.fit_transform(prb_df_encode['MAGIC_KEY'])


In [13]:
### drop purchase date and last purchase date column
#prb_df_encode = prb_df_encode.drop(columns=['MAGIC_KEY'])

In [14]:
prb_df_encode.duplicated().sum()

0

In [15]:
prb_df_encode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5379 entries, 0 to 15323
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   MAGIC_KEY                5379 non-null   object        
 1   BOX_ID                   5378 non-null   float64       
 2   QUALITY                  5378 non-null   object        
 3   DELIVERY_OPTION          5378 non-null   object        
 4   MILK                     5378 non-null   float64       
 5   MEAT                     5378 non-null   float64       
 6   UNIT_PRICE               5378 non-null   float64       
 7   PURCHASE_DATE            5378 non-null   datetime64[ns]
 8   BOX_COUNT                5378 non-null   float64       
 9   LAST_PURCHASE_DATE       5378 non-null   datetime64[ns]
 10  DAYS_FROM_LAST_PURCHASE  5378 non-null   float64       
 11  DAY                      5378 non-null   float64       
 12  MONTH                    5378 non-null

In [16]:
merged_df['MAGIC_KEY_ENCODED'] = le.fit_transform(merged_df['MAGIC_KEY'])


## splitting dataset for training and val set

In [17]:
len(merged_df)

2455723

In [18]:
train_size = int(.75 * len(merged_df))
train_size

1841792

In [19]:
sorted_df = merged_df.sort_values('PURCHASE_DATE')
train_df, val_df = sorted_df[:train_size], sorted_df[train_size:]

In [20]:
len(train_df), len(val_df)

(1841792, 613931)

In [21]:
val_df.PURCHASE_DATE.min(), val_df.PURCHASE_DATE.max()

(Timestamp('2019-01-30 00:00:00'), Timestamp('2019-02-28 00:00:00'))

In [22]:
train_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,PURCHASE_DATE,MAGIC_KEY,BOX_COUNT,LAST_PURCHASE_DATE,...,02_LAST_PURCHASE_2019,TOTAL_MEAT,TOTAL_MILK,MEAT_PURCHASE_FREQUENCY,MILK_PURCHASE_FREQUENCY,TOTAL_BOX_COUNT,TOTAL_AMOUNT,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED
2333148,260,Standard,Delivery from Collection Point,10.0,1.8,17.98,2018-10-01,29339036B48,1.0,2018-10-26,...,0,4.0,18.0,1,1,2.0,35.96,236131,0,1
736244,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2CA943107B8,1.0,2018-12-21,...,0,3.8,8.0,1,1,2.0,31.96,1004932,0,0
736245,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2C00FF1E2DC,1.0,2018-11-18,...,0,3.1,24.0,1,1,2.0,31.96,562275,0,0
736246,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2C1A276B78F,1.0,2019-02-13,...,0,8.5,63.0,1,1,5.0,79.90,636275,0,0
736247,118,Premium,Delivery from Collection Point,8.0,1.3,15.98,2018-10-01,2BD8A103BA3,1.0,2019-02-22,...,1,5.7,37.0,3,2,4.0,63.92,421092,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815827,119,Premium,Delivery from Collection Point,8.0,1.5,15.98,2019-01-30,2C86DEB844E,1.0,2019-01-30,...,0,9.0,38.0,1,1,4.0,63.92,902204,0,0
815813,119,Premium,Delivery from Collection Point,8.0,1.5,15.98,2019-01-30,2BEE97414AF,1.0,2019-01-30,...,0,1.5,8.0,1,1,1.0,15.98,504249,0,0
2027135,246,Standard,Delivery from Collection Point,0.0,2.5,13.96,2019-01-30,2900C0144CB,1.0,2019-02-23,...,1,12.5,0.0,5,5,5.0,69.80,150679,0,1
815815,119,Premium,Delivery from Collection Point,8.0,1.5,15.98,2019-01-30,2BE5466FA31,1.0,2019-01-30,...,0,1.5,8.0,1,1,1.0,15.98,474191,0,0


# train, test, val split

In [23]:
#train_df.info()

In [24]:
### drop purchase date and last purchase date column
train_df = train_df.drop(columns=['PURCHASE_DATE', 'LAST_PURCHASE_DATE', 'MAGIC_KEY'])
val_df = val_df.drop(columns=['PURCHASE_DATE', 'LAST_PURCHASE_DATE', 'MAGIC_KEY'])
test_df = prb_df_encode.drop(columns=['PURCHASE_DATE', 'LAST_PURCHASE_DATE', 'MAGIC_KEY'])

In [25]:
train_df.shape, val_df.shape, test_df.shape

((1841792, 30), (613931, 30), (5379, 30))

In [26]:
train_df.columns

Index(['BOX_ID', 'QUALITY', 'DELIVERY_OPTION', 'MILK', 'MEAT', 'UNIT_PRICE',
       'BOX_COUNT', 'DAYS_FROM_LAST_PURCHASE', 'DAY', 'MONTH', 'YEAR',
       '10_FIRST_PURCHASE_2018', '10_LAST_PURCHASE_2018',
       '11_FIRST_PURCHASE_2018', '11_LAST_PURCHASE_2018',
       '12_FIRST_PURCHASE_2018', '12_LAST_PURCHASE_2018',
       '01_FIRST_PURCHASE_2019', '01_LAST_PURCHASE_2019',
       '02_FIRST_PURCHASE_2019', '02_LAST_PURCHASE_2019', 'TOTAL_MEAT',
       'TOTAL_MILK', 'MEAT_PURCHASE_FREQUENCY', 'MILK_PURCHASE_FREQUENCY',
       'TOTAL_BOX_COUNT', 'TOTAL_AMOUNT', 'MAGIC_KEY_ENCODED',
       'DELIVERY_ENCODED', 'QUALITY_ENCODED'],
      dtype='object')

### declaring input and target cols for train, val, test set

In [27]:
input_cols = ['BOX_ID',  'MILK', 'MEAT', 'UNIT_PRICE',
       'BOX_COUNT', 'DAYS_FROM_LAST_PURCHASE', 'DAY', 'MONTH', 'YEAR',
       '10_FIRST_PURCHASE_2018', '10_LAST_PURCHASE_2018',
       '11_FIRST_PURCHASE_2018', '11_LAST_PURCHASE_2018',
       '12_FIRST_PURCHASE_2018', '12_LAST_PURCHASE_2018',
       '01_FIRST_PURCHASE_2019', '01_LAST_PURCHASE_2019', '02_LAST_PURCHASE_2019', 'TOTAL_MEAT',
       'TOTAL_MILK', 'MEAT_PURCHASE_FREQUENCY', 'MILK_PURCHASE_FREQUENCY',
       'TOTAL_BOX_COUNT', 'TOTAL_AMOUNT', 'MAGIC_KEY_ENCODED',
       'DELIVERY_ENCODED', 'QUALITY_ENCODED']

In [28]:

target_col = '02_FIRST_PURCHASE_2019'

In [29]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

In [30]:
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

In [31]:
test_inputs = test_df[input_cols].copy()
# Test data does not have targets

### Imputing missing numeric values

In [32]:
from sklearn.impute import SimpleImputer

In [33]:
imputer = SimpleImputer(strategy = 'mean').fit(merged_df[input_cols])

In [34]:
train_inputs[input_cols] = imputer.transform(train_inputs[input_cols])
val_inputs[input_cols] = imputer.transform(val_inputs[input_cols])
test_inputs[input_cols] = imputer.transform(test_inputs[input_cols])

### checking the null values

In [35]:
test_inputs[input_cols].isna().sum()

BOX_ID                     0
MILK                       0
MEAT                       0
UNIT_PRICE                 0
BOX_COUNT                  0
DAYS_FROM_LAST_PURCHASE    0
DAY                        0
MONTH                      0
YEAR                       0
10_FIRST_PURCHASE_2018     0
10_LAST_PURCHASE_2018      0
11_FIRST_PURCHASE_2018     0
11_LAST_PURCHASE_2018      0
12_FIRST_PURCHASE_2018     0
12_LAST_PURCHASE_2018      0
01_FIRST_PURCHASE_2019     0
01_LAST_PURCHASE_2019      0
02_LAST_PURCHASE_2019      0
TOTAL_MEAT                 0
TOTAL_MILK                 0
MEAT_PURCHASE_FREQUENCY    0
MILK_PURCHASE_FREQUENCY    0
TOTAL_BOX_COUNT            0
TOTAL_AMOUNT               0
MAGIC_KEY_ENCODED          0
DELIVERY_ENCODED           0
QUALITY_ENCODED            0
dtype: int64

# Scaling Numeric Features

In [36]:
#from sklearn.preprocessing import MinMaxScaler

In [37]:
#scaler = MinMaxScaler().fit(merged_df[input_cols])

In [38]:
# train_inputs[input_cols] = scaler.transform(train_inputs[input_cols])
# val_inputs[input_cols] = scaler.transform(val_inputs[input_cols])
# test_inputs[input_cols] = scaler.transform(test_inputs[input_cols])

In [39]:
#val_inputs.describe().loc[['min', 'max']]

In [40]:
#test_inputs.head()

## X_train, X_val, X_test

In [41]:
X_train = train_inputs
X_val = val_inputs
X_test = test_inputs

In [42]:
X_train.head()

,BOX_ID,MILK,MEAT,UNIT_PRICE,BOX_COUNT,DAYS_FROM_LAST_PURCHASE,DAY,MONTH,YEAR,10_FIRST_PURCHASE_2018,...,02_LAST_PURCHASE_2019,TOTAL_MEAT,TOTAL_MILK,MEAT_PURCHASE_FREQUENCY,MILK_PURCHASE_FREQUENCY,TOTAL_BOX_COUNT,TOTAL_AMOUNT,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED
2333148,260.0,10.0,1.8,17.98,1.0,25.0,1.0,10.0,2018.0,1.0,...,0.0,4.0,18.0,1.0,1.0,2.0,35.96,236131.0,0.0,1.0
736244,118.0,8.0,1.3,15.98,1.0,81.0,1.0,10.0,2018.0,1.0,...,0.0,3.8,8.0,1.0,1.0,2.0,31.96,1004932.0,0.0,0.0
736245,118.0,8.0,1.3,15.98,1.0,48.0,1.0,10.0,2018.0,1.0,...,0.0,3.1,24.0,1.0,1.0,2.0,31.96,562275.0,0.0,0.0
736246,118.0,8.0,1.3,15.98,1.0,135.0,1.0,10.0,2018.0,1.0,...,0.0,8.5,63.0,1.0,1.0,5.0,79.90,636275.0,0.0,0.0
736247,118.0,8.0,1.3,15.98,1.0,144.0,1.0,10.0,2018.0,1.0,...,1.0,5.7,37.0,3.0,2.0,4.0,63.92,421092.0,0.0,0.0


## model training

In [43]:
from sklearn.tree import DecisionTreeRegressor


In [44]:
# Assuming model is DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=42)

In [45]:
%%time
# Fit the model
model.fit(X_val, val_targets)

CPU times: total: 1.8 s
Wall time: 1.8 s


DecisionTreeRegressor(random_state=42)

### Evaluation

Let's evaluate the decision tree using the accuracy score.

In [46]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Calculate RMSE
#rmse = sqrt(mean_squared_error(true_values, predicted_values))


In [47]:
val_preds = model.predict(X_val)
val_preds

array([0., 0., 0., ..., 0., 0., 0.])

In [48]:
#train_preds

In [49]:
# Calculate RMSE
#rmse = sqrt(mean_squared_error(train_preds, train_targets )) 

In [50]:
#round(rmse)

### Baseline ML model

Let's train a simple `LinearRegression` model, with no customization.

In [51]:
from sklearn.linear_model import LinearRegression

In [52]:
linreg = LinearRegression()

In [53]:
linreg.fit(X_train, train_targets)

LinearRegression()

In [54]:
train_preds = linreg.predict(X_train)
train_preds

array([-0.06189657,  0.46820612,  0.08249561, ..., -0.04629395,
        0.08122536,  0.5225819 ])

In [55]:
mean_squared_error(train_preds, train_targets, squared=False)

0.27572323304336577

In [56]:
linreg.fit(X_val, val_targets)

LinearRegression()

In [57]:
val_preds = linreg.predict(X_val)
val_preds

array([ 0.17243127, -0.0459225 ,  0.14525391, ..., -0.45460025,
        0.05524442,  0.11176674])

In [58]:
mean_squared_error(val_preds, val_targets, squared=False)

0.18181790537203718

## Step 6 - Pick a strategy, train a model & tune hyperparameters

<img src="https://i.imgur.com/aRuE5mw.png" width="640">


### Systematically Exploring Modeling Strategies

Scikit-learn offers the following cheatsheet to decide which model to pick.

![](https://scikit-learn.org/stable/_static/ml_map.png)


Here's the general strategy to follow:

- Find out which models are applicable to the problem you're solving.
- Train a basic version for each type of model that's applicable
- Identify the modeling approaches that work well and tune their hypeparameters
- [Use a spreadsheet](https://docs.google.com/spreadsheets/d/1S42lnklqdj4hS3BohffyvZVJOFBZJ61VDiHLMuf2zBw/edit?usp=sharing) to keep track of your experiments and results.

Let's define a function `try_model`, which takes a model, then performs training and evaluation.

In [59]:
def try_model(model):
    # Fit the model
    model.fit(X_train, train_targets)
    
    # Generate predictions
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    #test_preds = model.predict(X_test)
    
    # Compute RMSE
    train_rmse = sqrt(mean_squared_error(train_targets, train_preds, squared=False))
    val_rmse = sqrt(mean_squared_error(val_targets, val_preds, squared=False))
    #test_rmse = sqrt(mean_squared_error(test_targets, test_preds, squared=False))
    return train_rmse, val_rmse #, test_rmse

### Linear Models

Read about linear models here: https://scikit-learn.org/stable/modules/linear_model.html

In [60]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor

In [61]:
try_model(LinearRegression())

(0.5250935469450808, 0.6204613467203018)

In [62]:
try_model(Ridge())

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.22881e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


(0.5250935474957902, 0.6204808008610366)

In [63]:
try_model(Lasso())

(0.6213476700090411, 0.7955787869210338)

In [64]:
try_model(ElasticNet())

(0.6206658132872623, 0.7991091773336084)

In [65]:
try_model(SGDRegressor())

C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


(4046487615.5271835, 4122148057.1047864)

### Tree Based Models

* Decision trees: https://scikit-learn.org/stable/modules/tree.html
* Random forests and gradient boosting: https://scikit-learn.org/stable/modules/ensemble.html

In [66]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [67]:
tree = DecisionTreeRegressor(random_state=42)
try_model(tree)

(0.0, 0.843811337303383)

In [68]:
# plt.figure(figsize=(40, 20))
# plot_tree(tree, max_depth=10, filled=True, feature_names=input_cols);

In [69]:
from sklearn.ensemble import RandomForestRegressor

In [70]:
 %%time
rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators= 500,   max_leaf_nodes = 2**20, max_features=30)
 try_model(rf)

CPU times: total: 3h 24min 2s
Wall time: 20min 26s


(0.15662886718918076, 0.8422320105217138)

In [71]:
rf.feature_importances_

array([3.01310978e-03, 8.86950975e-04, 2.15742483e-03, 1.38710944e-03,
       4.96529238e-05, 4.10115972e-01, 5.64638080e-02, 1.08098575e-01,
       3.46752124e-02, 1.03012397e-02, 6.91053107e-03, 1.93819875e-03,
       1.98757546e-03, 1.97063048e-03, 5.56082043e-03, 1.52838629e-02,
       5.87602310e-02, 1.16395354e-01, 1.48375089e-02, 9.08918697e-03,
       3.47908335e-03, 4.20629600e-03, 1.12205441e-01, 3.13142749e-03,
       1.63368474e-02, 5.65550436e-04, 1.92399114e-04])

In [72]:
importance_df = pd.DataFrame({ 
    'feature': input_cols, 
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)
importance_df.head(10)

,feature,importance
5,DAYS_FROM_LAST_PURCHASE,0.410116
17,02_LAST_PURCHASE_2019,0.116395
22,TOTAL_BOX_COUNT,0.112205
7,MONTH,0.108099
16,01_LAST_PURCHASE_2019,0.058760
6,DAY,0.056464
8,YEAR,0.034675
24,MAGIC_KEY_ENCODED,0.016337
15,01_FIRST_PURCHASE_2019,0.015284
18,TOTAL_MEAT,0.014838


### Making a submission on Kaggle

If you're participating in a Kaggle competition, you can generate a submission CSV file and make a submission to check your score on the test set.

In [73]:
test_preds = rf.predict(X_test)
test_preds

array([0., 0., 0., ..., 0., 0., 1.])

In [74]:
test_df['MEAT_PURCHASE'] = test_preds

In [75]:
test_df.MEAT_PURCHASE.min(), test_df.MEAT_PURCHASE.max()

(0.0, 1.0)

In [79]:
test_df

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,BOX_COUNT,DAYS_FROM_LAST_PURCHASE,DAY,MONTH,...,TOTAL_MEAT,TOTAL_MILK,MEAT_PURCHASE_FREQUENCY,MILK_PURCHASE_FREQUENCY,TOTAL_BOX_COUNT,TOTAL_AMOUNT,MAGIC_KEY_ENCODED,DELIVERY_ENCODED,QUALITY_ENCODED,MEAT_PURCHASE
0,56.0,Premium,Delivery from Collection Point,10.0,3.6,15.00,1.0,5.0,23.0,2.0,...,18.0,50.0,5.0,5.0,16.0,240.00,1334,0.0,0.0,0.0
5,132.0,Premium,Delivery from Collection Point,9.0,1.8,19.98,1.0,64.0,1.0,10.0,...,4.0,20.0,1.0,1.0,2.0,39.96,4064,0.0,0.0,0.0
7,162.0,Premium,Delivery from Collection Point,10.0,2.2,23.98,1.0,0.0,28.0,2.0,...,6.6,31.0,3.0,2.0,3.0,71.94,4428,0.0,0.0,0.0
10,56.0,Premium,Delivery from Collection Point,10.0,3.6,15.00,1.0,5.0,23.0,2.0,...,31.8,102.0,7.0,8.0,27.0,405.00,4453,0.0,0.0,0.0
20,162.0,Premium,Delivery from Collection Point,10.0,2.2,23.98,1.0,0.0,4.0,2.0,...,6.6,31.0,3.0,2.0,3.0,71.94,4398,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15319,77.0,Premium,Delivery from Collection Point,0.0,2.5,11.96,1.0,0.0,15.0,12.0,...,2.5,0.0,1.0,1.0,1.0,11.96,3853,0.0,0.0,0.0
15320,8.0,Premium,Home Delivery - CoD,10.0,0.0,12.18,1.0,0.0,25.0,11.0,...,0.0,10.0,1.0,1.0,1.0,12.18,3854,1.0,0.0,0.0
15321,3.0,Premium,Home Delivery - CoD,0.0,2.4,11.96,1.0,0.0,31.0,1.0,...,2.4,0.0,1.0,1.0,1.0,11.96,3855,1.0,0.0,0.0
15322,17.0,Premium,Home Delivery - CoD,10.0,1.8,12.98,1.0,0.0,25.0,10.0,...,1.8,10.0,1.0,1.0,1.0,12.98,3856,1.0,0.0,0.0


In [76]:
#columns_to_keep = ['MAGIC_KEY', 'X']
#sub = sub_df[columns_to_keep]


In [77]:
#sub = sub.rename(columns={'X': 'MEAT'})


In [78]:
#sub.to_csv('Submission.csv', index=None)